In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

f2 = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
# X_sequence_val, X_feature_val, Y_val = pickle.load(f2)
f2.close()



from keras_preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
total_X_image_train = []
total_Y_train = []
def build_channel(data):
    X = []
    for image in data:
        X.append(image.reshape(128, 128, 1))
    return X
for i in range(0,10):
    f = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    # X_sequence_train, X_feature_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    X_image_train = np.array(build_channel(X_image_train))
    Y_train = np.array(Y_train)-1
    Y_train = keras.utils.to_categorical(Y_train, num_classes=11)

    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    total_X_image_train.append(X_image_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
X_image_val = np.array(build_channel(X_image_val))
Y_val = np.array(Y_val)-1


Y_val = keras.utils.to_categorical(Y_val, num_classes=11)

2022-11-18 16:52:36.039845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 16:52:36.119928: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
from mix_models import rnn_with_feature

X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=860,
    decay_rate=0.85)

for i in range(0,10):
    model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=10, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


In [ ]:
from mix_models import cnn_with_feature_3

X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=860,
    decay_rate=0.85)

for i in range(0,10):
    model = cnn_with_feature_3(sequence_shape=(None, 3),image_shape=(128, 128, 1), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging_only_image_0.60_0.30_0.10/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging_only_image_0.60_0.30_0.10/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i], 'image':total_X_image_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=60, 
        validation_data=(X_val, Y_val), callbacks=callbacks)

In [ ]:
best_acc_list = [6,5,4,3,5,8,5,4,7,8]
min_loss_list = [6,6,4,8,7,8,5,6,5,8]
for i in range(0,10):
    loaded_model = keras.models.load_model('models/bagging_before_image_0.60_0.30_0.10/\
rnn_with_feature%d/model_%d'%(i,min_loss_list[i]), compile=False)
    loaded_model.save_weights('models/bagging_before_image_weights/rnn_weights_min_loss%d.h5'%i)

In [7]:
from mix_models import multi_input_model
X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val, 'image':X_image_val}
for i in range(0,10):
    model = multi_input_model(sequence_shape=(None, 3), image_shape=(128, 128, 1), 
                        feature_shape=(2,), num_classes=11, cnn_lr_lamd=10)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),

    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath='models/bagging_after_image_0.60_0.30_0.10/mix_model%d/model_{epoch}'%i,
      save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging_after_image_0.60_0.30_0.10/mix_model%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i],
              'image':total_X_image_train[i]}
    model.load_weights('models/bagging_before_image_weights/rnn_weights_min_loss%d.h5'%i, by_name=True)
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=40, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


NameError: name 'total_X_image_train' is not defined